In [7]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates


from QUANTAXIS.QAUtil import DATABASE
from QUANTAXIS.QAUtil import  trade_date_sse
from QUANTAXIS.QAUtil.QADate_trade import (
    QA_util_get_pre_trade_date,
    QA_util_get_next_trade_date,
    QA_util_if_tradetime
)

import inspect
import ind.Alpha101 as toys

from sklearn import linear_model

# %load_ext autoreload
# %autoreload 2
# %aimport tools.Cacher

# def assemble_stocks_by_codes(codes):
#     files = list(map(lambda x:x+'_train_qfq',l))
#     return pd.concat(list(map(lambda file:load_cache(file,cache_type=CACHE_TYPE.STOCK),files))).sort_index(level=0)

def pretreate_data(data):
    returns = smpl.get_current_return(data,'close')
    returns.name = 'returns'
    ret_forward = smpl.get_forward_return(data,'close')
    ret_forward.name = 'ret_forward'
    # {'Open', 'cap', 'close', 'high', 'ind', 'low', 'returns', 'volume', 'vwap'}
    data = pd.concat([data, returns, ret_forward], axis=1)
    data = data.assign(vwap=data.amount/(data.volume*100))
    data.rename(columns = {"open":"Open",'market_value':'cap','industry':'ind'}, inplace=True)
    data['cap']=data['cap']/data['close'] # 数据取出来的是市值


    close_ind = pretreat.neutralize(data.close, data['ind'],categorical=['ind'])
    close_ind.name = 'close_ind'
    vwap_ind = pretreat.neutralize(data.vwap, data['ind'],categorical=['ind'])
    vwap_ind.name = 'vwap_ind'
    high_ind = pretreat.neutralize(data.high, data['ind'],categorical=['ind'])
    high_ind.name = 'high_ind'
    low_ind = pretreat.neutralize(data.low, data['ind'],categorical=['ind'])
    low_ind.name = 'low_ind'
    volume_ind = pretreat.neutralize(data.volume, data['ind'],categorical=['ind'])
    volume_ind.name = 'volume_ind'

    adv20 = excute_for_multidates(data.volume, lambda x:x.rolling(20).agg('mean'), level=1)
    adv20 = pd.concat([adv20,data['ind']],axis=1).dropna()
    adv20_ind = pretreat.neutralize(adv20.volume, adv20['ind'],categorical=['ind'])
    adv20_ind.name = 'adv20_ind'

    adv40 = excute_for_multidates(data.volume, lambda x:x.rolling(40).agg('mean'), level=1)
    adv40 = pd.concat([adv40, data['ind']],axis=1).dropna()
    adv40_ind = pretreat.neutralize(adv40.volume, adv40['ind'],categorical=['ind'])
    adv40_ind.name = 'adv40_ind'

    adv81 = excute_for_multidates(data.volume, lambda x:x.rolling(81).agg('mean'), level=1)
    adv81 = pd.concat([adv81, data['ind']],axis=1).dropna()
    adv81_ind = pretreat.neutralize(adv81.volume, adv81['ind'],categorical=['ind'])
    adv81_ind.name = 'adv81_ind'

    co_mixed = ((data.close * 0.60733) + (data.Open * (1 - 0.60733)))
    co_mixed_ind = pretreat.neutralize(co_mixed, data['ind'],categorical=['ind'])
    co_mixed_ind.name = 'co_mixed_ind'

    oh_mixed = ((data.Open * 0.868128) + (data.high * (1 - 0.868128)))
    oh_mixed_ind = pretreat.neutralize(oh_mixed, data['ind'],categorical=['ind'])
    oh_mixed_ind.name = 'oh_mixed_ind'

    lv_mixed = ((data.low * 0.721001) + (data.vwap * (1 - 0.721001)))
    lv_mixed_ind = pretreat.neutralize(lv_mixed, data['ind'],categorical=['ind'])
    lv_mixed_ind.name = 'lv_mixed_ind'

    return pd.concat([data, close_ind, vwap_ind, low_ind, high_ind, volume_ind, adv20_ind, adv40_ind, adv81_ind, co_mixed_ind, oh_mixed_ind, lv_mixed_ind], axis=1)




from tools import QAAdapter as qaadapter
from factorBase import FactorBase

class alpha101(FactorBase):

    def calc(self, data) -> pd.DataFrame:
        import inspect
        import ind.Alpha101 as a101
        from base.JuUnits import excute_for_multidates
        """

        the example is just a day datasource, u can use the min data to generate a day-frequence factor

        the factor should be in day frequence 
        """
        fun_name = self.factor_name.replace('101_', '')
        params = inspect.signature(getattr(a101, fun_name)).parameters.keys()
        factor = excute_for_multidates(data, lambda x: getattr(a101, fun_name)(*[x[param].copy() for param in params]) ,level=1)
        factor.columns = ['factor']

        factor = pd.DataFrame(factor)
        factor = factor.replace([-np.inf, np.inf], 0).fillna(value=0)
        factor.columns = ['factor']

        return factor.reset_index().dropna()


In [11]:
# ids = [11,24,38,41,42,47,57,69,80,82,83,88,93,97]
# ids = [11,24]

# l = smpl.get_codes_from_blockname('沪深300', sse='all')
# df_all =  assemble_stocks_by_codes(l) #文件已合并，不再适用
# df_treated = pretreate_data(df_all)
from tools import QAAdapter as qaadapter
import QUANTAXIS as QA
import tools.Sample_Tools as smpl

from base.Constants import trade_date_sse
gap = 90
end_date = '2023-07-21'
start_date = trade_date_sse[trade_date_sse.index(end_date)-gap]
# stock_list  = qaadapter.QA_adapter_get_code_from_block('sw_l1', '国防军工')
stock_list = smpl.get_codes_by_market(sse='all',only_main=True,filter_st=True)

data = QA.QA_fetch_stock_day_adv(stock_list, start_date, end_date).data
data = smpl.add_marketvalue_industry(data)

# #读取已经缓存的复权数据，并预处理
smpl.optimize_data_type(data)
df_treated = pretreate_data(data)

ids = np.arange(1,101)
np.random.shuffle(ids)

def generate_alpha101_factors(fun_ids, df=None):

    from qaenv import (clickhouse_ip, clickhouse_password, clickhouse_port,
                      clickhouse_user)
    for i in fun_ids:
        
        fun_name = 'alpha101_'+str(i)
        print(fun_name)
        factor_base = alpha101(fun_name, host=clickhouse_ip, port=clickhouse_port, user=clickhouse_user, password=clickhouse_password)
        factor_base.update_to_database(df)


worker=6
task = task_chunk_split(ids, worker)
results = parallal_task(worker, generate_alpha101_factors, task, df=df_treated)


Now in the main code. Process name is: base.JuUnits
base.JuUnits, subpid:41825  pid:41685


  0%|          | 0/6 [00:00<?, ?it/s]

alpha101_85
['alpha101_1', 'alpha101_10', 'alpha101_100', 'alpha101_11', 'alpha101_12', 'alpha101_13', 'alpha101_14', 'alpha101_15', 'alpha101_16', 'alpha101_17', 'alpha101_18', 'alpha101_19', 'alpha101_2', 'alpha101_20', 'alpha101_21', 'alpha101_22', 'alpha101_23', 'alpha101_24', 'alpha101_25', 'alpha101_26', 'alpha101_27', 'alpha101_28', 'alpha101_29', 'alpha101_3', 'alpha101_30', 'alpha101_31', 'alpha101_32', 'alpha101_33', 'alpha101_34', 'alpha101_35', 'alpha101_36', 'alpha101_37', 'alpha101_38', 'alpha101_39', 'alpha101_4', 'alpha101_40', 'alpha101_41', 'alpha101_42', 'alpha101_43', 'alpha101_44', 'alpha101_45', 'alpha101_46', 'alpha101_47', 'alpha101_48', 'alpha101_49', 'alpha101_5', 'alpha101_50', 'alpha101_51', 'alpha101_52', 'alpha101_53', 'alpha101_54', 'alpha101_55', 'alpha101_56', 'alpha101_57', 'alpha101_58', 'alpha101_59', 'alpha101_6', 'alpha101_60', 'alpha101_61', 'alpha101_62', 'alpha101_63', 'alpha101_64', 'alpha101_65', 'alpha101_66', 'alpha101_67', 'alpha101_68', 'a